<a href="https://colab.research.google.com/github/vitaliy-sharandin/psychotherapy-model/blob/main/src/fine-tuning/psy_ai_huggingface_model_registry_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psy-AI
Psy-AI is a Llama3.1-8b model instruction fine-tuned on depression dataset and is meant to help improve mental well-being.

In future iterations it is meant to be trained on multiple philosophical and psychological datasets in order to provide multifaceted answers to complex mental health issues.



# Datasets

Philosophy datasets (* for future training)
* https://www.kaggle.com/datasets/christopherlemke/philosophical-texts
* https://www.workwithdata.com/object/philosophy-science-complete-a-text-on-traditional-problems-schools-thought-book-by-edwin-h-c-hung-0000
* https://www.kaggle.com/datasets/christopherlemke/philosophy-authors-writings-german
* https://www.workwithdata.com/object/philosophical-inquiries-an-introduction-to-problems-philosophy-book-by-nicholas-rescher-0000
* https://www.workwithdata.com/object/roman-stoicism-book-by-edward-vernon-arnold-1857
* https://www.workwithdata.com/object/wisdom-energy-basic-buddhist-teachings-book-by-thubten-yeshe-1935

Psychology and mental health datasets

* Kaggle Depression data for chatbot https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot
* Kaggle Psychometrics dataset https://www.kaggle.com/discussions/general/304994
* Psychometric tests dataset https://ieee-dataport.org/documents/psychometric-tests-dataset
* Psychometric NLP https://paperswithcode.com/dataset/psychometric-nlp
* Reddit mental health dataset https://zenodo.org/record/3941387
* Reddit mental disorders identification https://www.kaggle.com/datasets/kamaruladha/mental-disorders-identification-reddit-nlp
* Kaggle Mental Health Conversational Data https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
* Kaggle Mental Health FAQ for Chatbot https://www.kaggle.com/narendrageek/mental-health-faq-for-chatbot/code
* A human consciousness questionnaire dataset https://data.mendeley.com/datasets/69p62ksdh6
* paperswithcode Self-reported Mental Health Diagnoses https://paperswithcode.com/dataset/smhd
* paperswithcode Mental Health Summarization Dataset https://paperswithcode.com/dataset/mentsum
* HuggingFace psychology dataset https://huggingface.co/datasets/samhog/psychology-10k


In [13]:
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

import warnings
warnings.filterwarnings('ignore')

# Dataset instruction transformation

Depression dataset with 51 q&a entries was taken for mvp to see if model is indeed training.

In [14]:
depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

First, we modify our dataset to Alpaca format and create two datasets. One - for testing and evaluation and second one for inference, where responses are not available in formatted prompt.

In [15]:
def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

In [16]:
formatted_depression_dataset_train = depression_dataset.map(formatting_func_train)
formatted_depression_dataset_test = depression_dataset.map(formatting_func_test)

# Model load

We are loading our Llama3.1 model in 4bit quantized form as well as applying Lora for peft training.

In [17]:
model_name = "NousResearch/Hermes-3-Llama-3.1-8B"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():


  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0}
  )

  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.14s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

# Model instruction fine-tuning

Here we are defining inference function which returns bleu, rouge and f1 metrics after comparison of predicted and reference responses. My tests have shown that standard trainer `compute_metrics` method is quite inefficient and is not quite suitable for instruction fine-tuning during manual observations or generated results.

In [18]:
target_folder = 'target/'
model_trained_checkpoint = target_folder + 'model-trained-checkpoint'
model_merged = target_folder + 'final-model'

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):

    supervised_finetuning_trainer = SFTTrainer(model=model,
                                                train_dataset=train_dataset,
                                                eval_dataset=eval_dataset,
                                                args=TrainingArguments(
                                                    output_dir=target_folder + "training_results",
                                                    num_train_epochs=20,
                                                    per_device_train_batch_size=8,
                                                    per_device_eval_batch_size=8,
                                                    gradient_accumulation_steps=4,
                                                    optim="paged_adamw_8bit",
                                                    save_steps=500,
                                                    logging_steps=30,
                                                    learning_rate=1e-4,
                                                    weight_decay=0.001,
                                                    fp16=False,
                                                    bf16=False,
                                                    max_grad_norm=0.3,
                                                    max_steps=-1,
                                                    warmup_ratio=0.1,
                                                    group_by_length=True,
                                                    lr_scheduler_type="cosine",
                                                    report_to="none" if only_evaluate else "mlflow",
                                                ),
                                                compute_metrics=metrics_func,
                                                data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

    if not only_evaluate:
        mlflow.set_experiment('psy-model-finetuning')

        if mlflow.active_run():
            mlflow.end_run()

        mlflow.start_run()

        supervised_finetuning_trainer.train()
        supervised_finetuning_trainer.model.save_pretrained(model_trained_checkpoint)
        tokenizer.save_pretrained(model_trained_checkpoint)

        mlflow.log_params(supervised_finetuning_trainer.args.to_dict())
        mlflow.log_params({"model_name": model_name})
        mlflow.log_params({"model_trained_checkpoint": model_trained_checkpoint})

        results = supervised_finetuning_trainer.evaluate()
        mlflow.log_metrics({
            "eval_loss": results["eval_loss"],
            "eval_bleu": results["eval_bleu"],
            "eval_rouge1": results["eval_rouge1"],
            "eval_rouge2": results["eval_rouge2"],
            "eval_rougeL": results["eval_rougeL"],
            "eval_f1": results["eval_f1"]
        })

        mlflow.transformers.log_model(
                    transformers_model={"model": supervised_finetuning_trainer.model, "tokenizer": tokenizer},
                    task="text-generation",
                    artifact_path=model_trained_checkpoint
                    )

        mlflow.end_run()

    return supervised_finetuning_trainer.evaluate()

## Evaluating base model

In [19]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train["train"],
    "eval_dataset": formatted_depression_dataset_train["train"],
    "metrics_func": bleu_rouge_f1_metrics,
    "only_evaluate": True
}

fine_tune(**params)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
 57%|█████▋    | 4/7 [00:06<00:05,  1.93s/it]

KeyboardInterrupt: 

Evaluation metrics are quite low, as out model is not responding in a manner expected.

## Fine-tuning base and evaluating tuned model

In [8]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train["train"],
    "eval_dataset": formatted_depression_dataset_train["train"],
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)

100%|██████████| 20/20 [01:07<00:00,  3.37s/it]


{'train_runtime': 67.373, 'train_samples_per_second': 15.14, 'train_steps_per_second': 0.297, 'train_loss': 2.564951515197754, 'epoch': 10.0}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
100%|██████████| 7/7 [00:05<00:00,  1.39it/s]
Device set to use cuda:0
2025/02/08 20:42:59 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor and config will be saved, but the base model weights will not and reference to the HuggingFace Hub repository will be logged instead.
2025/02/08 20:42:59 INFO mlflow.transformers: Skipping saving pretrained model weights to disk as the save_pretrained argumentis set to False. The reference to the HuggingFace Hub repository NousResearch/Hermes-3-Llama-3.1-8B will be logged instead.
2025/02/08 20:43:00 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local

🏃 View run grandiose-zebra-697 at: http://127.0.0.1:8080/#/experiments/493982170954058773/runs/671b4d14a22f4e2895f07d18d159d884
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/493982170954058773


100%|██████████| 7/7 [00:04<00:00,  1.49it/s]


{'eval_loss': 2.333407163619995,
 'eval_bleu': 0.21936057569970438,
 'eval_rouge1': 0.5736305654891698,
 'eval_rouge2': 0.24307351400775307,
 'eval_rougeL': 0.49981907963761385,
 'eval_f1': 0.3173602441409385,
 'eval_runtime': 5.3924,
 'eval_samples_per_second': 9.458,
 'eval_steps_per_second': 1.298,
 'epoch': 10.0}

We can see that all scores are significantly improved after training.

# Saving fine-tuned model with adapters to Hugging Face

Now we are able to merge our model with saved peft adapters and push it to Hugging Face repo.

In [9]:
mlflow.autolog(disable=True)

tuned_model = AutoPeftModelForCausalLM.from_pretrained(
    model_trained_checkpoint,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map = {"": 0},
    quantization_config=BitsAndBytesConfig(load_in_4bit=True)
)
tokenizer = AutoTokenizer.from_pretrained(model_trained_checkpoint)

merged_model = tuned_model.merge_and_unload()

merged_model.save_pretrained(model_merged, safe_serialization=True)
tokenizer.save_pretrained(model_merged)

# token=''
# merged_model.push_to_hub("vitaliy-sharandin/wiseai", token=token)
# tokenizer.push_to_hub("vitaliy-sharandin/wiseai", token=token)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.10s/it]


('target/final-model\\tokenizer_config.json',
 'target/final-model\\special_tokens_map.json',
 'target/final-model\\tokenizer.json')

# Gradio demo in Hugging Face Spaces

https://huggingface.co/spaces/vitaliy-sharandin/WiseAI

## Local launch
Requirements:
* Choose free T4 GPU in Colab environment settings
* Run `pip install` and `imports` cells at the beginning before launching Gradio.

In [10]:
# model = AutoModelForCausalLM.from_pretrained(
#       'vitaliy-sharandin/wiseai',
#       load_in_8bit=True,
#       device_map = {"": 0}
#   )
# tokenizer = AutoTokenizer.from_pretrained('vitaliy-sharandin/wiseai')

# pipe = pipeline('text-generation', model=model,tokenizer=tokenizer)

# def generate_text(instruction, input):
#   if not instruction.strip():
#     return str('The instruction field is required.')

#   if instruction.strip() and input.strip():
#     input_prompt = (f"Below is an instruction that describes a task. "
#       "Write a response that appropriately completes the request.\n\n"
#       "### Instruction:\n"
#       f"{instruction}\n\n"
#       "### Input:\n"
#       f"{input}\n\n"
#       f"### Response: \n")
#   else :
#     input_prompt = (f"Below is an instruction that describes a task. "
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction:\n"
#         f"{instruction}\n\n"
#         f"### Response: \n")
#   result = pipe(input_prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
#   return result[:str(result).find("###")]

# iface = gr.Interface(fn=generate_text, inputs=[gr.Textbox(label="Instruction"),
#                                                gr.Textbox(label="Additional Input")],
#                      outputs=gr.Textbox(label="Response"))
# iface.launch()